In [1]:
! pip install mlflow boto3 awscli optuna xgboost imbalanced-learn
! pip install optuna


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import mlflow
# step 1 : Set up the mlflow tracking server 
mlflow.set_tracking_uri("http://ec2-51-20-65-11.eu-north-1.compute.amazonaws.com:5000/")

In [3]:
# Set or create an experiment
mlflow.set_experiment("Exp 5 - ML Algos with HP Tuning")

<Experiment: artifact_location='s3://youtube-recommender-exp-bucket/5', creation_time=1763818486726, experiment_id='5', last_update_time=1763818486726, lifecycle_stage='active', name='Exp 5 - ML Algos with HP Tuning', tags={}>

In [4]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,f1_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from imblearn.under_sampling import RandomUnderSampler
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

c:\Users\mukti\Desktop\youtube_sentiment_analysis\youtube_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
df = pd.read_csv(r'C:\Users\mukti\Desktop\youtube_sentiment_analysis\youtube_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [6]:
# Step 1: (Optional) Remapping - skipped since not strictly needed for Multimodal naive bayes

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

## Step 2: Train-test split BEFORE sampling

X_train_raw,X_test_raw,y_train,y_test = train_test_split(
        df['clean_comment'],df['category'],
        test_size=0.2,
        random_state=42,
        stratify=df['category']
)

# Step 3: BOW vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 1000  # Set max_features to 1000

vectorizer = CountVectorizer(ngram_range=ngram_range, max_features=max_features)

X_train_vec = vectorizer.fit_transform(X_train_raw)
X_test_vec = vectorizer.transform(X_test_raw)

# Step 4: Apply Random Undersampler to train only
rus = RandomUnderSampler(random_state=42)
X_train_res, y_train_res = rus.fit_resample(X_train_vec, y_train)



#Function to log results in MLflow
def log_mlflow(model_name,model,X_train,X_test,y_train,y_test):
    with mlflow.start_run():
        #log the model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_Undersampling_BOW_Trigram")
        mlflow.set_tag("experiment_type", "Algorithm comparision")

        #log algorith name as parameter
        mlflow.log_param("algo_name",model_name)

        #Train model
        model.fit(X_train,y_train)
        y_pred =model.predict(X_test)

        # log accuracy
        accuracy = accuracy_score(y_test,y_pred)
        mlflow.log_metric("accuracy",accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for Logistic regression
def objective_mnb(trial):
    alpha = trial.suggest_float('alpha', 1e-4, 1.0, log=True)  # Tuning the smoothing parameter


    model = MultinomialNB(alpha=alpha)
    model.fit(X_train_res, y_train_res)
    preds = model.predict(X_test_vec)

    return accuracy_score(y_test,preds)


# Step 7: Run Optuna for Multinomial Naive Bayes, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_mnb, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = MultinomialNB(alpha=best_params['alpha'])

    # Log the best model with MLflow, passing the algo_name as "Multinomial Naive Bayes"
    log_mlflow("Multinomial Naive Bayes", best_model, X_train_res, X_test_vec, y_train_res, y_test)

# Run the experiment for Multinomial Naive Bayes
run_optuna_experiment()




[I 2025-11-22 19:17:32,442] A new study created in memory with name: no-name-0206e730-1637-43ba-b2d8-7f9a83dc338e
[I 2025-11-22 19:17:32,462] Trial 0 finished with value: 0.6218464475657984 and parameters: {'alpha': 0.0008939664063735126}. Best is trial 0 with value: 0.6218464475657984.
[I 2025-11-22 19:17:32,469] Trial 1 finished with value: 0.6213009682258285 and parameters: {'alpha': 0.13568213279738248}. Best is trial 0 with value: 0.6218464475657984.
[I 2025-11-22 19:17:32,475] Trial 2 finished with value: 0.6218464475657984 and parameters: {'alpha': 0.002253891216659514}. Best is trial 0 with value: 0.6218464475657984.
[I 2025-11-22 19:17:32,481] Trial 3 finished with value: 0.6169371335060685 and parameters: {'alpha': 0.7016754187699239}. Best is trial 0 with value: 0.6218464475657984.
[I 2025-11-22 19:17:32,488] Trial 4 finished with value: 0.6218464475657984 and parameters: {'alpha': 0.0007093454159568675}. Best is trial 0 with value: 0.6218464475657984.
[I 2025-11-22 19:17:32

🏃 View run Multinomial Naive Bayes_Undersampling_BOW_Trigram at: http://ec2-51-20-65-11.eu-north-1.compute.amazonaws.com:5000/#/experiments/5/runs/acd621a19cd3462aa083da967ea5993e
🧪 View experiment at: http://ec2-51-20-65-11.eu-north-1.compute.amazonaws.com:5000/#/experiments/5
